In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES']='5'

In [2]:
from datasets import load_dataset
dataset = load_dataset("zera09/finance_chat", split = "test")

In [3]:
# from PIL import Image
# import random
# from tqdm.notebook import tqdm
# # Set random seed for reproducibility
# random.seed(42)

# # Determine indices to set image_path to None (50% of dataset)
# total_samples = len(dataset)
# exclude_indices = set(random.sample(range(total_samples), total_samples // 2))



# # Modify dataset in place
# def modify_image_path(example, index):
#     if index in exclude_indices:
#         example["image_path"] = 'None'
#     return example

# # Apply modification
# dataset = dataset.map(modify_image_path, with_indices=True)

# def format_data(sample):
#     return [
#         {
#             "role": "system",
#             "content": [{"type": "text", "text": 'You are a finacnial assistant.'}],
#         },
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "image",
#                     "image":Image.open('/home/sarmistha/Sarmistha/ECML/'+sample['image_path']).resize((255,255)),
#                 },
#                 {
#                     "type": "text",
#                     "text": sample["query"],
#                 },
#             ],
#         },
#     ],sample['answer']

# def format_data_text(sample):
#     return [
#         {
#             "role": "system",
#             "content": [{"type": "text", "text": 'You are a finacnial assistant.'}],
#         },
#         {
#             "role": "user",
#             "content": [
#                 {
#                     "type": "text",
#                     "text": sample["query"],
#                 },
#             ],
#         },
        
#     ],sample['answer']



# labels =[]
# test_dataset = [ ]
# for sample in tqdm(dataset):
#     if os.path.exists('/home/sarmistha/Sarmistha/ECML/'+sample['image_path']):
#         conv,label = format_data(sample)
#         test_dataset.append(conv)
#         labels.append(label)
#     else:
#         conv,label = format_data_text(sample)
#         test_dataset.append(conv)
#         labels.append(label)

In [4]:
import os
import random
from PIL import Image
from tqdm.notebook import tqdm

# Set random seed for reproducibility
random.seed(42)

# Determine indices to set image_path to None (50% of dataset)
total_samples = len(dataset)
exclude_indices = set(random.sample(range(total_samples), total_samples // 2))

# Modify dataset in place
def modify_image_path(example, index):
    if index in exclude_indices:
        example["image_path"] = 'None'
    return example

# Apply modification
dataset = dataset.map(modify_image_path, with_indices=True)

# Define formatting functions
def format_data(sample):
    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": 'You are a financial assistant.'}],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": Image.open('/home/sarmistha/Sarmistha/ECML/' + sample['image_path']).resize((255, 255)),
                },
                {
                    "type": "text",
                    "text": sample["query"],
                },
            ],
        },
    ], sample['answer']

def format_data_text(sample):
    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": 'You are a financial assistant.'}],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": sample["query"],
                },
            ],
        },
    ], sample['answer']

# Separate samples into those with and without image modalities
image_samples = []
text_only_samples = []

for sample in tqdm(dataset):
    image_path = '/home/sarmistha/Sarmistha/ECML/' + sample['image_path']
    if os.path.exists(image_path):
        image_samples.append(sample)
    else:
        text_only_samples.append(sample)

# Select 50 from each category
selected_image_samples = random.sample(image_samples, min(50, len(image_samples)))
selected_text_only_samples = random.sample(text_only_samples, min(50, len(text_only_samples)))

# Create test dataset and labels
test_dataset = []
labels = []

for sample in selected_image_samples:
    conv, label = format_data(sample)
    test_dataset.append(conv)
    labels.append(label)

for sample in selected_text_only_samples:
    conv, label = format_data_text(sample)
    test_dataset.append(conv)
    labels.append(label)


  0%|          | 0/515 [00:00<?, ?it/s]

In [5]:
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

model_id = "google/gemma-3-4b-it"
model =Gemma3ForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

processor = AutoProcessor.from_pretrained(model_id)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [6]:
# adapter_path = "zera09/gemma3-fin-chat_missing"
# model.load_adapter(adapter_path)

In [7]:
from qwen_vl_utils import process_vision_info


def generate_text_from_sample(model, processor, sample, max_new_tokens=256, device="cuda"):
    # Prepare the text input by applying the chat template
    inputs = processor.apply_chat_template(
        sample, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)
    
    input_len = inputs["input_ids"].shape[-1]
    
    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=150, do_sample=False)
        generation = generation[0][input_len:]
    
    decoded = processor.decode(generation, skip_special_tokens=True)
    return decoded 

In [8]:
def generate_text_for_all(model, processor, test_dataset):
    outputs = []
    for sample in tqdm(test_dataset):
        output = generate_text_from_sample(model, processor, sample)
        outputs.append(output)
    return outputs

# Example usage:
all_outputs = generate_text_for_all(model, processor, test_dataset)


  0%|          | 0/100 [00:00<?, ?it/s]

/home/sarmistha/miniconda3/envs/slomvlm/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/sarmistha/miniconda3/envs/slomvlm/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [9]:
import json

ans = {'pred': all_outputs, 'label': labels}

with open('gemma_missing_zero.json', 'w') as f:
    json.dump(ans, f)
